## Using the ArcticDB DataFrame Database for Tick Storage

In [ ]:
!conda install -c conda-forge arcticdb -y

In [ ]:
from IPython.display import display, Markdown
import time
import pytz
import datetime as dt
import pandas as pd
import arcticdb as adb

In [ ]:
from thetadata import (
    ThetaClient,
    OptionRight,
    StreamMsg,
    StreamMsgType,
    StreamResponseType,
)

Initialize ArcticDB and create a library for trades

In [ ]:
arctic = adb.Arctic("lmdb://arcticdb_options")
lib = arctic.get_library("trades", create_if_missing=True)

Function to get the trade datetime

In [ ]:
def get_trade_datetime(today, ms_of_day):
    return today + dt.timedelta(milliseconds=ms_of_day)

Function to get the number of days to expiration

In [ ]:
def get_days_to_expiration(today, expiration):
    return (expiration - today).days

Callback function to handle received messages

In [ ]:
def callback(msg):
    today = dt.datetime.now(pytz.timezone("US/Eastern")).replace(
        hour=0, minute=0, second=0, microsecond=0
    )

    if msg.type == StreamMsgType.TRADE:
        trade_datetime = get_trade_datetime(today, msg.trade.ms_of_day)
        expiration = pd.to_datetime(msg.contract.exp).tz_localize("US/Eastern")
        days_to_expiration = get_days_to_expiration(today, expiration)
        symbol = msg.contract.root
        trade = {
            "root": symbol,
            "expiration": expiration,
            "days_to_expiration": days_to_expiration,
            "is_call": msg.contract.isCall,
            "strike": msg.contract.strike,
            "size": msg.trade.size,
            "trade_price": msg.trade.price,
            "exchange": str(msg.trade.exchange.value[1]),
            "bid_size": msg.quote.bid_size,
            "bid_price": msg.quote.bid_price,
            "ask_size": msg.quote.ask_size,
            "ask_price": msg.quote.ask_price,
        }
        trade_df = pd.DataFrame(trade, index=[trade_datetime])
        if symbol in lib.list_symbols():
            lib.update(symbol, trade_df, upsert=True)
        else:
            lib.write(symbol, trade_df)

Function to start streaming all trades

In [ ]:
def stream_all_trades():
    client = ThetaClient(username="YOUR_USERNAME", passwd="YOUR_PASSWORD")

    client.connect_stream(callback)

    req_id = client.req_full_trade_stream_opt()

    response = client.verify(req_id)

    if client.verify(req_id) != StreamResponseType.SUBSCRIBED:
        raise Exception("Unable to stream.")

    time.sleep(120)  # Stream for 2 minutes

    print("Cancelling stream...")

    client.remove_full_trade_stream_opt()

Start streaming trades

In [ ]:
stream_all_trades()

Defragment symbols if necessary

In [ ]:
for symbol in lib.list_symbols():
    if lib.is_symbol_fragmented(symbol):
        lib.defragment_symbol_data(symbol)

List the symbols in the library

In [ ]:
lib.list_symbols()

Read all the data for one of the symbols (e.g., QQQ)

In [ ]:
qqq = lib.read("QQQ").data
display(qqq)

Use the query builder to find options with tight spread

In [ ]:
q = adb.QueryBuilder()
filter = (q.ask_price - q.bid_price) < 0.05
q = q[filter]
data = lib.read("QQQ", query_builder=q).data

In [ ]:
display(data)

Use the query builder to find options with more than 1 day to expiration

In [ ]:
q = adb.QueryBuilder()
filter = q.days_to_expiration > 1
q = q[filter].groupby("expiration").agg({"bid_size": "sum", "ask_size": "sum"})
data = lib.read("QQQ", query_builder=q).data.sort_index()

In [ ]:
display(data)

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.